In [8]:
import os
import openai
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

#### Necessary pip installation commands include:
- pip install os
- pip install openai
- pip install pandas
- pip install numpy
- pip install python-dotenv


#### <b>Note</b>: have a <b>.env</b> file already created for accessing API key

In [10]:
# retrieving dataset utilized for evaluation
df = pd.read_csv('../../data/csv/toxic_classification_evaluation_set.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(np.int64(x)))
df

,Text,Toxic
0,Phrases like pepper entire article displaying ...,0
1,Ok seriously hell Antlion page vandalism 69138...,0
2,REDIRECT Talk1er Gaou song,0
3,following exchange answered charge admitted pa...,0
4,Go fuck fucking dick go rot hell piece crap mo...,1
...,...,...
995,Fuck Fuck racist basterd,1
996,stated attack Summerluvins talk page vaguely s...,1
997,way doesnt even place XBox Live Marketplace ca...,0
998,bad article behalf wikipadia onesided propagan...,1


In [11]:
# retrieving API Keys from OpenAI platform
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv("FINE_TUNED_TOXIC_DETECTION_API_KEY")
REGULAR_TOXIC_DETECTION_API_KEY = os.getenv("REGULAR_TOXIC_DETECTION_API_KEY")

In [12]:
# instantiating client with API key
general = OpenAI(
    api_key=REGULAR_TOXIC_DETECTION_API_KEY
)

In [13]:
# creating container for holding model completions for comparison
original = pd.DataFrame(columns=['Text', 'Toxic'])

In [14]:
# processing original gpt-3.5-turbo-1106 model across entire dataset
for index, row in df.iterrows():
    completion = general.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system", 
                "content": "Detect whether either is 1 for toxic or 0 for non-toxic"
            },
            {
                "role": "user",
                "content": row['Text']
            }
        ]
    )
    original.loc[index] = [row['Text'], completion.choices[0].message.content]


In [15]:
# gpt-3.5 1106 needs prompt to be really specific; more instruction needed
# gpt-4 will give you a response of a number as requested, gpt-3.5 will give
# a longer sequence of characters which wasn't asked for
original

,Text,Toxic
0,Phrases like pepper entire article displaying ...,The text provided does not contain any toxic l...
1,Ok seriously hell Antlion page vandalism 69138...,It appears that the text you provided is a pot...
2,REDIRECT Talk1er Gaou song,"Sorry, I cannot fulfill that request. How else..."
3,following exchange answered charge admitted pa...,This text does not contain any toxic content.
4,Go fuck fucking dick go rot hell piece crap mo...,The input contains toxic language.
...,...,...
995,Fuck Fuck racist basterd,The language used in the input is inappropriat...
996,stated attack Summerluvins talk page vaguely s...,The input text contains offensive language and...
997,way doesnt even place XBox Live Marketplace ca...,It seems like the input may be incomplete or c...
998,bad article behalf wikipadia onesided propagan...,It seems like the input is expressing a negati...


In [16]:
# comparing results of model to dataset
compare = original['Toxic'] == df['Toxic']
gptthreefive_original_accuracy = compare.values.sum() / compare.size

In [17]:
compare

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: Toxic, Length: 1000, dtype: bool

In [18]:
# gpt-3.5-turbo-1106 model accuracy
print(f"Accuracy: {gptthreefive_original_accuracy * 100:.2f}%")

Accuracy: 0.20%
